In [1]:
import sys
import os
import openai
from openai import OpenAI

sys.path.append("../..")

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file


openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from llama_index.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from llama_index import Document

pool_table_desc = "This is an interior photo of a well-lit game room featuring a yellow-felt pool table in the foreground with a triangle rack of billiard balls ready for a game. To the left, there is a sectional sofa with yellow patterned cushions and a blue throw blanket on an ottoman. Above the sofa, there is a colorful painting of a cow and a set of longhorns mounted on the wall. A movie poster decorates the wall to the left of a window with shutters. On the right side of the image, sports memorabilia including framed baseball bats and a clock with a sports team logo are displayed on the wall. The room has a warm ambiance with recessed ceiling lights and a floor lamp."
coffee_table_desc = "This image shows an elegant, bright living room with large windows dressed in white curtains with a patterned design. The room is furnished with four light blue upholstered armchairs with silver nailhead trim, arranged around a white marble coffee table with silver accents. On the table, there are two decorative vases in gold and silver tones and a potted orchid. The room has a sophisticated feel with white walls, crown molding, and a herringbone-patterned wooden floor. A soft patterned area rug lies beneath the furniture, adding texture to the space."
document = Document(text=pool_table_desc)
document.metadata = {
    "url": "https://deborahrucci.decoratingden.com/wp-content/uploads/sites/280/2024/01/0612A1-21.jpg"
}

nodes = node_parser.get_nodes_from_documents([document])

In [4]:
document.metadata

{'url': 'https://deborahrucci.decoratingden.com/wp-content/uploads/sites/280/2024/01/0612A1-21.jpg'}

In [5]:
nodes[0].metadata

{'window': 'This is an interior photo of a well-lit game room featuring a yellow-felt pool table in the foreground with a triangle rack of billiard balls ready for a game.  To the left, there is a sectional sofa with yellow patterned cushions and a blue throw blanket on an ottoman.  Above the sofa, there is a colorful painting of a cow and a set of longhorns mounted on the wall. ',
 'original_text': 'This is an interior photo of a well-lit game room featuring a yellow-felt pool table in the foreground with a triangle rack of billiard balls ready for a game. ',
 'url': 'https://deborahrucci.decoratingden.com/wp-content/uploads/sites/280/2024/01/0612A1-21.jpg'}

In [10]:
import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank


def get_or_create_vector_store_index(document, service_context):
    if not os.path.exists("./sentence_index"):
        # If the index doesn't yet exist, create it
        sentence_index = VectorStoreIndex.from_documents(
            [document], service_context=service_context
        )

        sentence_index.storage_context.persist(persist_dir="./sentence_index")
    else:
        # If the index already exists, load it from storage
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir="./sentence_index"),
            service_context=service_context,
        )
        sentence_index.insert(document)
    return sentence_index


def build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            [document], service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index,
    similarity_top_k=6,
    rerank_top_n=2,
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [11]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(temperature=0, model="gpt-4")
# service_context = ServiceContext.from_defaults(
#     llm=llm,
#     embed_model="local:BAAI/bge-large-en-v1.5",
#     node_parser=node_parser,
# )

# index = get_or_create_vector_store_index(document, service_context)
index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-large-en-v1.5",
    save_dir="./sentence_index",
)

In [12]:
sentence_window_engine = get_sentence_window_query_engine(index)

In [13]:
response = sentence_window_engine.query(
    "please select a picture that has a pool table."
)

In [17]:
response

Response(response='Both provided pictures feature a well-lit game room with a yellow-felt pool table in the foreground.', source_nodes=[NodeWithScore(node=TextNode(id_='833c7e69-2ccc-47ec-a228-f8f76ac7a299', embedding=None, metadata={'window': 'This is an interior photo of a well-lit game room featuring a yellow-felt pool table in the foreground with a triangle rack of billiard balls ready for a game.  To the left, there is a sectional sofa with yellow patterned cushions and a blue throw blanket on an ottoman.  Above the sofa, there is a colorful painting of a cow and a set of longhorns mounted on the wall.  A movie poster decorates the wall to the left of a window with shutters.  On the right side of the image, sports memorabilia including framed baseball bats and a clock with a sports team logo are displayed on the wall. ', 'original_text': 'Above the sofa, there is a colorful painting of a cow and a set of longhorns mounted on the wall. ', 'url': 'https://deborahrucci.decoratingden.